# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [6]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': [
        'customers',
        'orders',
        'products',
        'employees'
    ],
    'definition': [
        'Stores customer data: customer_id, name, email, phone, country.',
        'Stores orders data: order_id, customer_id, order_date, total_amount.',
        'Stores products data: product_id, name, category, price, stock_quantity.',
        'Stores employees data: employee_id, name, role, department, hire_date.'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  customers  Stores customer data: customer_id, name, email...
1     orders  Stores orders data: order_id, customer_id, ord...
2   products  Stores products data: product_id, name, catego...
3  employees  Stores employees data: employee_id, name, role...


In [7]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [8]:
print(text_tables)

customers: Stores customer data: customer_id, name, email, phone, country.
orders: Stores orders data: order_id, customer_id, order_date, total_amount.
products: Stores products data: product_id, name, category, price, stock_quantity.
employees: Stores employees data: employee_id, name, role, department, hire_date.


In [9]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [11]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which customers have placed the most orders?")

In [12]:
print(return_OAI(pqt1))

```json
{
    "tables": ["customers", "orders"]
}
```


In [13]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What are the top-selling products in the last 3 months?")

In [14]:
print(return_OAI(pqt3))

{
    "tables": ["orders", "products"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [15]:
# 📌 Estructura del prompt
prompt_question_tables = """
Given the following tables and their content definitions:

### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{question}
"""

# 📌 Pruebas con diferentes preguntas
user_queries = [
    "Which customers have placed the most orders?",
    "What are the top-selling products in the last 3 months?",
    "Which employees have been working for more than 5 years?",
]


In [16]:
for query in user_queries:
    formatted_prompt = prompt_question_tables.format(tables=text_tables, question=query)
    result = return_OAI(formatted_prompt)
    print(f"📝 **Query:** {query}\n📌 **Tables Identified:** {result}\n" + "-"*50)

📝 **Query:** Which customers have placed the most orders?
📌 **Tables Identified:** ```json
{
    "tables": ["customers", "orders"]
}
```
--------------------------------------------------
📝 **Query:** What are the top-selling products in the last 3 months?
📌 **Tables Identified:** {
    "tables": ["orders", "products"]
}
--------------------------------------------------
📝 **Query:** Which employees have been working for more than 5 years?
📌 **Tables Identified:** {
    "tables": ["employees"]
}
--------------------------------------------------
